# Example
of the last added functionalities in onlineSG

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import sys
sys.path.append('../')  # uncomment to add the path to the onlineSG folder 
# You have to change the path if you move this file.
# Alternatevely add the onlineSG folder to your PYTHONPATH
# e.g. On Linux add this line on .bashrc:
# export PYTHONPATH="<absolute-path-to-onlineSG>:PYTHONPATH"
# then you can use onlineSG library from everywhere

In [14]:
import source.game as game
import source.player as player
import source.environment as environment
import source.errors as errors
import source.parsers as parsers
import source.players.attackers as attackers
import source.players.base_defenders as base_defenders
import source.players.defenders as defenders
import source.runner as runner
import source.players.detection as detection 
import source.util as util

from source.util import *
import numpy as np
import pandas as pd
from importlib import *
import time

In [90]:
# if you change something in the files, don't forget to reload
reload(player)
reload(base_defenders)
reload(attackers)
reload(defenders)
reload(errors)
reload(runner)
reload(parsers)
reload(game)
#reload(environment)
reload(detection)
reload(util)

<module 'source.util' from '/home/onlinesg/onlineSG/source/util.py'>

[0.312405, 0.324887, 0.084612]

In [16]:
targets_dict = {i:[] for i in range(2,13)}
T = 15 #np.random.randint(2,10)
time_horizon =1000
while sum([len(targets_dict[x]) for x in range(2,13)]) < 9:
    targets = [round(x,3) for x in util.gen_norm_targets(T)]
    values = tuple((v, v) for v in targets)
    g = game.Game(values, time_horizon)
    g.attackers = [1]
    g.defenders = [0]
    s = util.support(g)
    if len(targets_dict[len(s)])==0:
        targets_dict[len(s)].append(s)

In [87]:
T = 3
P = 2
targets = translate(targets_dict[T][0])
distributions = []
for i in range(P):
    distributions.append(tuple(gen_distr(T)))
values = tuple((v, v) for v in targets)
g = game.Game(values, time_horizon)
#print(T, targets)
profiles = [attackers.StackelbergAttacker(g, 1)]
for d in distributions:
    #print(d)
    profiles.append(attackers.StochasticAttacker(g, 1, 1, *d))
agent = detection.HOLMES(g, 0, 1, 1)
attacker = profiles[0]
g.set_players([agent], [attacker], [p for p in profiles if p != attacker])

In [58]:
T = 3
P * P * T + T * T * T * P

66

In [59]:
2.16 * (456 / 66)

14.923636363636366

In [70]:
[p.distribution for p in e.game.profiles if p != e.game.players[1]]

[[0.18666680128528187,
  0.2751403007000473,
  0.3278924162835406,
  0.025855477191970767,
  0.18444500453915935],
 [0.057268665858167929,
  0.22328435876112757,
  0.25615541905852562,
  0.17377864495579171,
  0.28951291136638713]]

In [88]:
e = runner.Experiment(g)

In [84]:
start_time = time.time()
e.run_interaction()
print(game_str(e.game, lenght=7))
print(e.agent.belief)
print(e.game.strategy_history[-1])
print(time.time() - start_time)

□	 	 	
0.665	0.727	0.54	
 	 	△	
{<StochasticAttacker id:1 resources:1>: 0.0046744694796351741, <StackelbergAttacker id:1 resources:1>: 0.0, <StochasticAttacker id:1 resources:1>: 0.99532553052036488}
{0: (0.3643124030976371, 0.4185251004950875, 0.21716249640727533), 1: [0, 0, 1]}
2.069589853286743


In [89]:
e.run_interaction()

In [79]:
e.game.players[1].game.strategy_history[-1]

{0: (0.3690684156632389, 0.4282480801547355, 0.20268350418202563),
 1: [0, 0, 1]}

In [80]:
e.game.strategy_history[-1]

{0: (0.3690684156632389, 0.4282480801547355, 0.20268350418202563),
 1: [0, 0, 1]}

In [78]:
e.agent.profiles[0].game.strategy_history[-1]

{0: (0, 1, 0), 1: [0, 0, 1]}

In [65]:
e.game.strategy_history[-1] 

{0: (0.07000285365649891,
  0.30581066825329867,
  0.14416530551802176,
  0.3511386294556226,
  0.1288825431165581),
 1: [1, 0, 0, 0, 0]}

In [37]:
start_time = time.time()
e.run()
print(time.time() - start_time)

378.01570677757263


In [ ]:
start_time = time.time()

In [24]:
time_horizon = 100
targets = (1,2,3)
values = tuple((v, v) for v in targets)
distribution1 = (0,0.5,0.5)
distribution2 = (0,0.1,0.9)
g = game.Game(values, time_horizon)
#profiles = [attackers.StackelbergAttacker(g, 1, 1)] #,
profiles = [attackers.StochasticAttacker(g, 1, 1, *distribution1)]#, attackers.StochasticAttacker(g, 1, 1, *distribution2)] #,
agent = detection.StrategyAwareDetector(g, 0, 1)
attacker = attackers.StackelbergAttacker(g, 1, 1)
#attacker = attackers.StochasticAttacker(g, 1, 1, *distribution1)
g.set_players([agent], [attacker], profiles)

In [26]:
c = runner.Configuration(g, "concurrent", "prova", False)

In [25]:
rm -rf concurrent

In [27]:
c.run(3)

In [724]:
e = runner.Experiment(g)

In [756]:
e.run_interaction()
#print(game_str(e.game, lenght=7))
print(e.agent.belief, e.agent.detection)
print(e.game.strategy_history[-1])

[0, 0.9, 0.1]
0.0
False
0.5
False
0.5
False
{<StochasticAttacker id:1 resources:1>: 1.853020188851843e-17, <StackelbergAttacker id:1 resources:1>: 1.0} None
{0: [0.0, 0.39, 0.61], 1: [0, 1, 0]}


In [757]:
e.agent.tau

32

In [720]:
e.run()

[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
Fals

In [74]:
time_horizon = 100
T = 4
targets = (5,4,8,1)
values = tuple((v, v) for v in targets)
distribution1 = (0.17,0.8, 0.01, 0.02)
g = game.Game(values, time_horizon)

other_kinds = [attackers.StochasticAttacker(g, 1, 1, *distribution1)] #,
strategy_aware = attackers.StackelbergAttacker(g, 1, 1)
L = 2
agent = detection.StrategyAwareDetector.parse("sad1-0.01@stackelberg@stochastic_attacker1-0.17-0.8-0.01-0.02", g, 0) 
#agent = detection.HOLMES(g, 0, 1, strategy_aware, other_kinds, L)
# attacker = attackers.StackelbergAttacker(g, 1, 1)
attacker = attackers.StochasticAttacker(g, 1, 1, *distribution1)
g.set_players([agent], [attacker])

In [76]:
e1 = runner.Experiment(g)

In [77]:
e1.run_interaction()
print(game_str(e1.game, lenght=7))
print(e1.agent.belief)
print(e1.game.strategy_history[-1])

□	 	 	 	
5	4	8	1	
 	△	 	 	
{<StackelbergAttacker id:1 resources:1>: 0.0, <StochasticAttacker id:1 resources:1>: 1.0}
{0: [0.30934782608695643, 0.13543478260869565, 0.5552173913043479, 0.0], 1: [0.17, 0.8, 0.01, 0.02]}
